In [32]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [33]:
import torch
import sklearn

In [34]:
# seed 고정
torch.manual_seed(1)

In [35]:
# data load
train = pd.read_csv("/kaggle/input/2021-ai-quiz1-p5/train.csv")
test = pd.read_csv("/kaggle/input/2021-ai-quiz1-p5/test.csv")
submit = pd.read_csv("/kaggle/input/2021-ai-quiz1-p5/submit_sample.csv")

In [36]:
# 데이터 확인

print(train)
print(test)
print(submit)

In [37]:
## data 전처리 및 텐서 올리기
# 스케일 조정을 위한 Scaler

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

x_train = np.array(train.drop(['diagnosis'],axis=1))
x_test = np.array(test)
y_train = np.array(train['diagnosis'])

# scale 조정
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

x_train = torch.Tensor(x_train).cuda()
x_test = torch.Tensor(x_test).cuda()
y_train = torch.Tensor(y_train).cuda()

print(x_train.shape,y_train.shape)

In [38]:
# 학습 설정
w = torch.zeros((30,1),requires_grad=True,device="cuda")
b = torch.zeros(1,requires_grad=True,device="cuda")

optim = torch.optim.SGD([w,b],lr = 1e-1)
epochs= 2000
# 이진 분류 문제 이므로 BCELoss 사용
loss = torch.nn.BCELoss()

In [39]:
for epoch in range(epochs+1):
    optim.zero_grad()
    
    h = torch.sigmoid(x_train.matmul(w)+b)
    cost = loss(h,y_train.unsqueeze(1))
    
    cost.backward()
    optim.step()
    
    if epoch % 100 == 0:
        print(cost.item())

In [40]:
with torch.no_grad():
    predict = torch.sigmoid(x_test.matmul(w) +b)
    

predict = predict >= torch.Tensor([0.5]).cuda()
submit['diagnosis'] = predict.cpu().int().numpy()
submit.to_csv("submission.csv",index=False)
print(submit)

# try 
# lr = 1e-4 에 epoch 1000 cost 가 무수히 감소함
# lr = 1e-3 도 동일
# lr = 1e-2 도 동일
# lr = 1e-1 도 감소함 
# epoch 2000 에서 0.03 정도의 코스트에 고정됨 